<a href="https://colab.research.google.com/github/carlosdcorona/PySpark/blob/main/pySparkEDAClimateChange.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#pySpark EDA. Climate Change

### Carlos Daniel Corona Cortez A01656002
### October 10, 2022


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.3/spark-3.1.3-bin-hadoop2.7.tgz
!tar xf spark-3.1.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
#All imports
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.3-bin-hadoop2.7"

In [ ]:
import findspark
from pyspark import SparkContext, SparkConf,SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col, lit
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

In [ ]:
# create sparksession and sparkcontext
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
filepath = '/content/drive/Shareddrives/Infra DS Equipo 4/Homework 05/climate_change.csv'

spark = SparkSession \
    .builder \
    .getOrCreate()

sqlContext = SQLContext(sc)

In [ ]:
df = sqlContext.read.csv(filepath,header=True,inferSchema=True)
df.show(10)

+----+-----+------+------+-------+-------+-------+-------+---------+--------+-----+
|Year|Month|   MEI|   CO2|    CH4|    N2O| CFC-11| CFC-12|      TSI|Aerosols| Temp|
+----+-----+------+------+-------+-------+-------+-------+---------+--------+-----+
|1983|    5| 2.556|345.96|1638.59|303.677|191.324|350.113|1366.1024|  0.0863|0.109|
|1983|    6| 2.167|345.52|1633.71|303.746|192.057|351.848|1366.1208|  0.0794|0.118|
|1983|    7| 1.741|344.15|1633.22|303.795|192.818|353.725| 1366.285|  0.0731|0.137|
|1983|    8|  1.13|342.25|1631.35|303.839|193.602|355.633|1366.4202|  0.0673|0.176|
|1983|    9| 0.428|340.17| 1648.4|303.901|194.392|357.465|1366.2335|  0.0619|0.149|
|1983|   10| 0.002| 340.3|1663.79| 303.97|195.171|359.174|1366.0589|  0.0569|0.093|
|1983|   11|-0.176|341.53|1658.23|304.032|195.921|360.758|1366.1072|  0.0524|0.232|
|1983|   12|-0.176|343.07|1654.31|304.082|196.609|362.174|1366.0607|  0.0486|0.078|
|1984|    1|-0.339|344.05|1658.98| 304.13|197.219|363.359|1365.4261|  0.0451

In [ ]:
df.dtypes

[('Year', 'int'),
 ('Month', 'int'),
 ('MEI', 'double'),
 ('CO2', 'double'),
 ('CH4', 'double'),
 ('N2O', 'double'),
 ('CFC-11', 'double'),
 ('CFC-12', 'double'),
 ('TSI', 'double'),
 ('Aerosols', 'double'),
 ('Temp', 'double')]

In [ ]:
df.toPandas().isnull().sum()

Year        0
Month       0
MEI         0
CO2         0
CH4         0
N2O         0
CFC-11      0
CFC-12      0
TSI         0
Aerosols    0
Temp        0
dtype: int64

In [ ]:
df_omit_time = df.select([col for col in df.columns if col not in  ('Year','Month')])
df_omit_time.summary().show()

+-------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+
|summary|                MEI|               CO2|               CH4|               N2O|            CFC-11|            CFC-12|               TSI|           Aerosols|               Temp|
+-------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+
|  count|                308|               308|               308|               308|               308|               308|               308|                308|                308|
|   mean|0.27555519480519486|363.22675324675305|1749.8245129870143|312.39183441558424| 251.9730681818181|497.52478246753236|1366.0707590909085|0.01665714285714292| 0.2567759740259741|
| stddev| 0.9379184591591946|12.647124861976184| 46.05167823751526| 5.2251307296

##